In [1]:
from dotenv import load_dotenv
import os
from google import generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import pandas as pd
import numpy as np
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent


In [2]:
load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [3]:
file_path = r"C:\Users\LeeSeungYong\langchain\QA-Chatbot\sample\sample_superstore.xls"

In [4]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash-latest"
)

In [5]:
if file_path.endswith((".xls",".xlsx")):
    print("yes")
    data = pd.read_excel(file_path)

yes


In [6]:
db_file = "temp_database.db"
conn = sqlite3.connect(db_file)
data.to_sql('uploaded_data', conn, if_exists="replace", index = False)
conn.close()


In [7]:
conn.close()

In [8]:
data.head(2).T

,0,1
Row ID,1,2
Order ID,CA-2016-152156,CA-2016-152156
Order Date,2016-11-08 00:00:00,2016-11-08 00:00:00
Ship Date,2016-11-11 00:00:00,2016-11-11 00:00:00
Ship Mode,Second Class,Second Class
Customer ID,CG-12520,CG-12520
Customer Name,Claire Gute,Claire Gute
Segment,Consumer,Consumer
Country,United States,United States
City,Henderson,Henderson


In [9]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute(
    """select city, avg(sales)
    from uploaded_data
    group by city
    limit 5
    """
)

In [10]:
query_result = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

In [11]:
df = pd.DataFrame(data=query_result, columns=column_names)

In [12]:
df

,City,avg(sales)
0,Aberdeen,25.500000
1,Abilene,1.392000
2,Akron,129.999333
3,Albuquerque,158.582857
4,Alexandria,344.973125


In [13]:
db_uri = f"sqlite:///{db_file}"
db = SQLDatabase.from_uri(db_uri)

In [14]:
user_query = """
 11월 지역별 평균 매출액을 비교해줘
"""

In [15]:
# from pprint import pprint
# import json 
# from ast import literal_eval
# literal_eval(prompt_template.json())

In [18]:
from langchain import hub
from langchain.agents import create_react_agent

# Pull prompt (or define your own)
prompt_template = hub.pull(
    "langchain-ai/sql-agent-system-prompt"
    )
system_message = prompt_template.format(
    dialect="SQLite", 
    top_k=5
    )
toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm
    )

# Create agent
agent_executor = create_sql_agent(
    llm,
    db=db,
    verbose=True
    )
# Query agent
# example_query = "2016년 11월 지역별 평균 매출액을 비교해줘"
# events = agent_executor.invoke(example_query,)

c:\Users\LeeSeungYong\langchain\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
from pprint import pprint
pprint(agent_executor.get_prompts)

<bound method Runnable.get_prompts of AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty

In [26]:
type(events.get("output"))

str

In [17]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(
    llm,
    db
    )
response = chain.invoke({"question": example_query})
response

'Question: 2016년 11월 지역별 평균 매출액을 비교해줘\nSQLQuery: SELECT "Region", AVG("Sales") AS "Average Sales" FROM uploaded_data WHERE strftime(\'%Y\', "Order Date") = \'2016\' AND strftime(\'%m\', "Order Date") = \'11\' GROUP BY "Region"'

In [18]:
print(chain.get_prompts()[0].template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

In [19]:
query = response.split("SQLQuery: ")[-1]

In [20]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute(query)

In [21]:
column_names = [description[0] for description in cursor.description]
query_result = cursor.fetchall()

In [22]:
query_df = pd.DataFrame(data=query_result, columns=column_names)

In [37]:
data[(data["Order Date"].apply(lambda x : x.year) == 2016) & (data["Order Date"].apply(lambda x : x.month) == 3)].groupby("State")["Sales"].mean().sort_values(ascending = False)

State
Oklahoma          1287.450000
California         755.116286
Missouri           697.160000
Maryland           691.116667
Florida            492.133333
New York           439.407556
Tennessee          364.256000
Texas              320.552200
Delaware           251.354000
New Jersey         244.006000
Illinois           233.494909
Pennsylvania       191.286211
Washington         182.336000
Ohio               160.071400
New Hampshire      159.980000
Mississippi        150.630000
Georgia            130.188000
Arizona            118.875500
Colorado           116.139000
Michigan           110.522000
Oregon             107.175400
Connecticut        105.545000
Indiana             95.243333
Massachusetts       95.106000
North Carolina      93.808000
Virginia            91.400000
Nevada              58.580000
Minnesota           31.720000
Idaho               17.340000
Wisconsin           16.740000
Louisiana           12.960000
Name: Sales, dtype: float64